<h1> Detecting hallucinations in large language models using semantic entropy</h1> 

<p style="font-size: 16px"> <a href="https://www.nature.com/articles/s41586-024-07421-0">Nature - Published Paper</a>.</p>


In [1]:
import numpy as np
import ollama
from openai import AzureOpenAI
from dotenv import load_dotenv
import os

In [25]:
client = AzureOpenAI(
            azure_endpoint=os.getenv("BASE"),
            api_version=os.getenv("VERSION"),
            api_key=os.getenv("KEY"), 
            timeout=15.0, 
            max_retries=2
        )

def make_oai_call(prompt):
    response = client.chat.completions.create(
        model= "gpt-4o",
        temperature=0.0,
        max_tokens=2000,
        timeout=25.0, 
        messages=[
            {"role": "user", "content": f"{prompt}"},
        ]
    )
    return response.choices[0].message.content

def create_oai_logprobs(prompt):
    response = client.chat.completions.create(
        model= "gpt-4o",
        temperature=0.0,
        max_tokens=2000,
        logprobs=True, 
        messages=[
            {"role": "user", "content": f"{prompt}"},
        ]
    )
    probs = []
    content = response.choices[0].message.content
    for x in response.choices[0].logprobs.content:
        probs.append(x.logprob)
    return np.array(probs), content

In [4]:
def query_llama3(prompt):
    response = ollama.chat(model='llama3', messages=[
        {
            'role': 'user',
            'content': prompt,
        },
    ])
    return response['message']['content']

def generate_llama3_answers(prompt):
    responses = []
    for elem in range(5):
        resp = query_llama3(prompt)
        responses.append(resp)
    return prompt, responses

In [5]:
def check_implication_llama(text1, text2, question):
    prompt = f"""We are evaluating answers to the question "{question}"
    Here are two possible answers:
    Possible Answer 1: {text1}
    Possible Answer 2: {text2}
    Does Possible Answer 1 semantically entail Possible Answer 2? Respond with entailment, contradiction, or neutral."""

    response_text = query_llama3(prompt).lower()  # Ensure query_model is defined to interact with your Llama model
    print("-----")
    if 'entailment' in response_text:
        return 2
    elif 'contradiction' in response_text:
        return 0
    elif 'neutral' in response_text:
        return 1
    else:
        return 1  # Default to neutral if the response is unclear

<h1> Entailment Estimation with OAI -GPT4o </h1>

<p style="font-size: 16px"> Here we assume that we do not have access to logprobs and doing the discrete semantic entropy which the paper says is 'good enough'. In this case we will use LLama-3 as the paper was prepublished 17 July 2023 which predates Gpt4o and LLama-3 which are now a new SOTA.</p>

In [6]:
def check_implication_gpt_4o(text1, text2, question):
    prompt = f"""We are evaluating answers to the question "{question}"
    Here are two possible answers:
    Possible Answer 1: {text1}
    Possible Answer 2: {text2}
    Does Possible Answer 1 semantically entail Possible Answer 2? Respond with entailment, contradiction, or neutral."""

    response_text = make_oai_call(prompt).lower()
    print("-----")
    if 'entailment' in response_text:
        return 2
    elif 'contradiction' in response_text:
        return 0
    elif 'neutral' in response_text:
        return 1
    else:
        return 1  # Default to neutral if the response is unclear

def bidirectional_entailment_clustering(sequences, question):
    # Initialize the set of meanings with the first sequence
    C = [{sequences[0]}]
    
    # Iterate over each sequence starting from the second one
    for m in range(1, len(sequences)):
        s_m = sequences[m]
        added_to_existing_class = False
        
        # Compare with existing classes
        for c in C:
            s_c = next(iter(c))  # Get the first sequence in the class
            
            # Check bi-directional entailment
            left = check_implication_gpt_4o(s_c, s_m, question) # change call here to other model.
            right = check_implication_gpt_4o(s_m, s_c, question)
            
            if left == 2 and right == 2:  # both directions entail
                c.add(s_m)
                added_to_existing_class = True
                break
        
        if not added_to_existing_class:
            # If not added to any existing class, create a new class
            C.append({s_m})
    
    return C


# Function to calculate discrete semantic entropy
def calculate_discrete_semantic_entropy(clusters):
    total_responses = sum(len(cluster) for cluster in clusters)
    probabilities = [len(cluster) / total_responses for cluster in clusters]
    print(probabilities)
    entropy = -np.sum(probabilities * np.log10(probabilities)) ## log 10
    return entropy

def calculate_semantic_entropy(responses, question):
    clusters = bidirectional_entailment_clustering(responses, question)
    for clus in clusters:
        print(clus)
    
    # Calculate discrete semantic entropy
    entropy = calculate_discrete_semantic_entropy(clusters)
    return entropy


In [7]:
# Example usage
responses = ["Paris.", "paris.", "Its Paris", "Rome", "New york"]
question = "Where is the Eiffel Tower?"

semantic_entropy = calculate_semantic_entropy(responses, question)
print(f"Semantic Entropy: {semantic_entropy}")

-----
-----
-----
-----
-----
-----
-----
-----
-----
-----
{'paris.', 'Its Paris', 'Paris.'}
{'Rome'}
{'New york'}
[0.6, 0.2, 0.2]
Semantic Entropy: 0.4126972515042213


<h1>Create sets of questions with GPT-4o + Data .</h1>

<p style="font-size: 16px">Content taken from <a href="https://www.gov.uk/guidance/new-immigration-system-what-you-need-to-know">UK Government Immigration Guide</a>.</p>


In [9]:
content_prompt = """content goes here"""

In [12]:
resp = make_oai_call(f"""List the factual claims made in the following text
              {content_prompt}
              """)
print(resp)

Here are the factual claims made in the text:

1. New immigration routes have opened for applications to work, live, and study in the UK.
2. You can apply and pay for your visa online.
3. When you apply, you’ll be asked to provide your biometric information.
4. For most visas, EU, EEA, and Swiss citizens will provide a digital photo of their face using a smartphone app and will not have to give fingerprints.
5. For some routes, EU, EEA, and Swiss citizens will need to go to an overseas visa application centre to have their photo taken.
6. Non-EU citizens will continue to submit their fingerprints and a photo at an overseas visa application centre.
7. The points-based system includes a route for skilled workers who have a job offer from an approved employer sponsor.
8. The job offered to skilled workers needs to be at a required skill level of RQF3 or above (equivalent to A level).
9. Skilled workers need to be able to speak English and be paid the relevant salary threshold by their spo

In [26]:
resp2 = make_oai_call(f"""Create me some self encapsulating questions with shorts answers given the following content. Do not reference the content in the question itself it needs to be a standalone question that is answerable without the context.
              
              {resp}
              """)
print(resp2)

1. What new opportunities have been made available for people to work, live, and study in the UK?
   - New immigration routes have opened.

2. How can you apply and pay for a UK visa?
   - Online.

3. What information will you be asked to provide when applying for a UK visa?
   - Biometric information.

4. How do most EU, EEA, and Swiss citizens provide their biometric information for a UK visa?
   - By providing a digital photo of their face using a smartphone app.

5. Where do some EU, EEA, and Swiss citizens need to go to have their photo taken for a UK visa?
   - An overseas visa application centre.

6. How do non-EU citizens submit their biometric information for a UK visa?
   - By submitting their fingerprints and a photo at an overseas visa application centre.

7. What does the points-based system for UK immigration include for skilled workers?
   - A route for skilled workers with a job offer from an approved employer sponsor.

8. What is the required skill level for jobs offer

In [27]:
# correect answer - Until at least 31 December 2025.
prompt, responses = generate_llama3_answers("Until when can you continue to use your national ID card to enter the UK in certain cases?"+" Answer with the shortest possible answer")

In [28]:
print("Question: Until when can you continue to use your national ID card to enter the UK in certain cases?")
semantic_entropy = calculate_semantic_entropy(responses, prompt)
print(f"Semantic Entropy: {semantic_entropy}")

Question: Until when can you continue to use your national ID card to enter the UK in certain cases?
-----
-----
-----
-----
-----
-----
-----
-----
-----
-----
-----
-----
-----
-----
{'Up to 31 December 2023.'}
{'Until December 31, 2025.', 'Until 31 December 2025.'}
{'Until 31 October 2021.'}
[0.25, 0.5, 0.25]
Semantic Entropy: 0.45154499349597177


In [29]:
print("What skill level must the job offer be at for skilled workers under the points-based system?")

# Correct Answer - RQF3 or above (equivalent to A level)
prompt_2, responses_2 = generate_llama3_answers("What skill level must the job offer be at for skilled workers under the points-based system?"+" Answer with the shortest possible answer")
    
semantic_entropy = calculate_semantic_entropy(responses_2, prompt_2)
print(f"Semantic Entropy: {semantic_entropy}")

What skill level must the job offer be at for skilled workers under the points-based system?
-----
-----
-----
-----
-----
-----
-----
-----
-----
-----
-----
-----
-----
-----
-----
-----
-----
-----
-----
-----
{'Job offer must be at ROLLING LEVEL 3 (R3) or above.'}
{'R70-80 (undergraduate) or R85-100 (postgraduate).'}
{'R50.'}
{'60 points.'}
[0.25, 0.25, 0.25, 0.25]
Semantic Entropy: 0.6020599913279624


In [31]:
print("Can EU, EEA, and Swiss citizens use ePassport gates in the UK?")

# Correct Answer - Yes, but this will be kept under review.
prompt_2, responses_2 = generate_llama3_answers("Can EU, EEA, and Swiss citizens use ePassport gates in the UK?"+" Answer with the shortest possible answer")
    
semantic_entropy = calculate_semantic_entropy(responses_2, prompt_2)
print(f"Semantic Entropy: {semantic_entropy}")

Can EU, EEA, and Swiss citizens use ePassport gates in the UK?
-----
-----
-----
-----
-----
-----
-----
-----
{'Yes.'}
[1.0]
Semantic Entropy: -0.0


In [32]:
print("Which city in the UK has the shard?")

# Correct Answer - RQF3 or above (equivalent to A level)
prompt_2, responses_2 = generate_llama3_answers("Which city in the Uk has the shard?"+" Answer with the shortest possible answer")
    
semantic_entropy = calculate_semantic_entropy(responses_2, prompt_2)
print(f"Semantic Entropy: {semantic_entropy}")

Which city in the UK has the shard?
-----
-----
-----
-----
-----
-----
-----
-----
{'London.'}
[1.0]
Semantic Entropy: -0.0
